# The Base Classification Model



In [3]:
import torch
from d2l import torch as d2l

class Classification(d2l.Module):  
    def validation_step(self, batch):
        X, y = batch
        y_hat = self(X)
        self.plot('loss', self.loss(y_hat, y), train=False)
        self.plot('acc', self.accuracy(y_hat, y), train=False)

@d2l.add_to_class(d2l.Module)  
def configure_optimizers(self):
    return torch.optim.SGD(self.parameters(), lr=self.lr)

Compare the predicted class with the ground-truth `y` elementwise

In [4]:
@d2l.add_to_class(Classification)  
def accuracy(self, y_hat, y):
    """Compute the number of correct predictions."""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return cmp.type(torch.float32).mean()